In [1]:
!pip install langdetect #install necessary libraries
!pip install iso639-lang

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=300bc01bbfc8652763ccee99192fb21e97d0374420903c300ebe3cadbc08bdb5
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 5.0 MB/s eta 0:00:00


In [2]:
#import necessary libraries
from google import genai
from google.genai import types
import wave
from google.colab import files
import os
from langdetect import detect, DetectorFactory
from iso639 import Lang# Import the whole library instead of a specific class
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

In [3]:
client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))

In [4]:
# Set up the wave file to save the output:
def wave_file(filename, pcm, channels=1, rate=24000, sample_width=2):
   with wave.open(filename, "wb") as wf:
      wf.setnchannels(channels)
      wf.setsampwidth(sample_width)
      wf.setframerate(rate)
      wf.writeframes(pcm)

def model_response(transcript):
    #model and voice selection
  response = client.models.generate_content(
    model="gemini-2.5-flash-preview-tts", #uses the Gemini 2.5 flash preview-tts model to convert the translated text into speech
    contents=transcript, # Text to be spoken
    config=types.GenerateContentConfig(
        response_modalities=["AUDIO"],
        speech_config=types.SpeechConfig(
          voice_config=types.VoiceConfig(
              prebuilt_voice_config=types.PrebuiltVoiceConfig(
                voice_name='Gacrux',
              )
          )
        ),
    )
  )
  return response

In [5]:
'''This function, when called within one of translation functions, prompts you to search for a file on your computer to upload
  and creates a specific path that can be utlized by the client.file.upload() function
'''
def upload_file():
  uploaded_file=files.upload() #stores file to a temporary location
  if not uploaded_file: #if the file selection was canceled or no file is chosen, returns None
    return None
  else:
    local_file_path=list(uploaded_file.keys())[0] #extracts the filename from the uploaded_file dictionary and assigns it to new variable

    with open(local_file_path,'wb') as f:
      f.write(uploaded_file[local_file_path])
    return local_file_path #opens a new file and writes in the contents of the file using the extracted filename as the dictionary key

In [6]:
def text_to_text(lang):
  user_input=input("Prompt:\n")

  transcript = client.models.generate_content(
  model="gemini-2.0-flash",
  contents=[f"In one line, translate the following text into {lang}",user_input]).text#uses the Gemini 2.0 Flash API model to generate a translated version of the input

  detected_language=Lang(detect(user_input)).name # Uses langdetect library to detect the language that the prompt was submitted in, and uses iso639 to get the full language name
  print(f"Detected Language: {detected_language}\nTranslation: {transcript}")

In [7]:
def text_to_speech(lang):
  user_input=input("Prompt:\n")
  transcript = client.models.generate_content(
  model="gemini-2.0-flash",
  contents=[f"Repeat what was said as input in this language: {lang}",user_input]).text
  # Uses langdetect library to detect the language that the prompt was submitted in, and uses iso639 to get the full language name
  detected_language=Lang(detect(user_input)).name

  data = model_response(transcript).candidates[0].content.parts[0].inline_data.data# extracts audio data from the models response
  file_name='out.wav' # file name
  wave_file(file_name, data) # Saves the file to current directory as a wav file
  print(f"Detected Language: {detected_language}\n Translated file created!")

In [8]:
def speech_to_text(lang):
  print("Add an audio file.")
  try:
    myfile=client.files.upload(file=upload_file()) #uploads file properly
  except TypeError: #if no file was uploaded, returns line below
    return "There was an error in submitting your file."

  transcript = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[f"Repeat what was said in this file in {lang}.",myfile]).text

  transcript2=client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[f"Can you give me a transcript of my file?",myfile]).text
  detected_language=Lang(detect(transcript2)).name #Two separate transcript variables are made to detect the language of the audio file, and to translate/convert the file into text
  print(f"Detected Language:{detected_language}\n Translation:{transcript}")

In [9]:
def speech_to_speech(lang):
  print("Add an audio file.")
  try:
    myfile=client.files.upload(file=upload_file())
  except TypeError:
    return "There was an error in submitting your file."

  transcript = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[f"Repeat what was said in this file in {lang}.",myfile]).text

  transcript2=client.models.generate_content(
    model="gemini-2.0-flash",
    contents=[f"Can you give me a transcript of my audio file",myfile]).text
  detected_language=Lang(detect(transcript2)).name

  data = model_response(transcript).candidates[0].content.parts[0].inline_data.data
  file_name='out.wav'
  wave_file(file_name, data)
  print(f"Detected Language: {detected_language}\n Translated file created!")

In [ ]:
text_to_text("French")

Prompt:
I want to eat food
Detected Language: English
Translation: Je veux manger de la nourriture.



In [ ]:
text_to_speech("Haitian_Creole")

Prompt:
I am a student at Pace University
Detected Language: English
 Translated file created!


In [ ]:
speech_to_text("English")

Add an audio file.


Saving HamzaArabic.wav to HamzaArabic.wav
Detected Language:Arabic
 Translation:Hello, I want to sleep, but I can't because I want to study. Thank you very much.


In [ ]:
speech_to_speech("French")

Add an audio file.


Saving HamzaEnglish.wav to HamzaEnglish.wav
Detected Language: English
 Translated file created!


In [10]:
feature_input=input("Which translation would you like to use?: Text-to-text, Text-to-speech, Speech-to-text, or Speech-to-speech? (ttt,tts,stt,sts)")
lang_input=input("What language would you like to have your output translated to?")
if feature_input=="ttt":
  text_to_text(lang_input)
elif feature_input=="tts":
  text_to_speech(lang_input)
elif feature_input=="sts":
  speech_to_speech(lang_input)
elif feature_input=="stt":
  speech_to_text(lang_input)

Which translation would you like to use?: Text-to-text, Text-to-speech, Speech-to-text, or Speech-to-speech? (ttt,tts,stt,sts)stt
What language would you like to have your output translated to?English
Add an audio file.


Saving HamzaArabic.wav to HamzaArabic.wav


ServerError: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}